# Twin Test: Base vs Instruct Models

**Paper #3 - Nature vs Nurture Hypothesis**

**Date:** 2026-01-05

**Purpose:** Test whether thermodynamic profiles (Gain) are "genetic traits" that persist through fine-tuning (RLHF).

---

## The Hypothesis

```
NATURE (Architecture) → Determines BASE LEVEL (Gain magnitude)
NURTURE (RLHF)       → Modulates but does NOT invert
```

**Prediction:**
- If Gemma-Base explodes (Gain > 2.0), Gemma-Instruct should also explode
- If Mistral-Base is neutral (Gain ≈ 1.0), Mistral-Instruct should also be neutral
- RLHF may AMPLIFY the trait, but not REVERSE it

---

## Twin Pairs to Test

| Family | Base Model | Instruct Model | Expected Base Gain |
|--------|------------|----------------|--------------------|
| Mistral | mistralai/Mistral-7B-v0.1 | mistralai/Mistral-7B-Instruct-v0.2 | ~1.11 (Neutral) |
| Gemma | google/gemma-7b | google/gemma-7b-it | ~2.31 (Explosion) |
| LLaMA 3.1 | meta-llama/Llama-3.1-8B | meta-llama/Llama-3.1-8B-Instruct | ~1.48 (Expansion) |

---

## Success Criteria

**Nature dominates if:**
- Base and Instruct have the SAME SIGN relative to 1.0
- Correlation between Base Gain and Instruct Gain > 0.8

**Nurture dominates if:**
- RLHF changes the sign (e.g., Base > 1.0 but Instruct < 1.0)
- Correlation < 0.5

In [ ]:
# Cell 1: Setup with ROBUST HF Token Handling
!pip install -q transformers accelerate scipy seaborn huggingface_hub

import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import AutoModelForCausalLM, AutoTokenizer
from scipy.stats import entropy, pearsonr, spearmanr, ttest_rel
import gc
import json
from datetime import datetime
from google.colab import files
import os
import warnings
warnings.filterwarnings('ignore')

# ============================================================
# HF TOKEN HANDLING - ROBUST VERSION
# ============================================================
HF_TOKEN = None

# Method 1: Try Colab secrets (primary method)
try:
    from google.colab import userdata
    HF_TOKEN = userdata.get('HF_TOKEN')
    if HF_TOKEN:
        print(f"✓ HF_TOKEN loaded from Colab secrets")
        print(f"  Token starts with: {HF_TOKEN[:10]}...")
except Exception as e:
    print(f"  Colab secrets method failed: {e}")

# Method 2: Try environment variable as fallback
if not HF_TOKEN:
    HF_TOKEN = os.environ.get('HF_TOKEN') or os.environ.get('HUGGINGFACE_TOKEN')
    if HF_TOKEN:
        print(f"✓ HF_TOKEN loaded from environment variable")

# Method 3: Explicit login to HuggingFace Hub
if HF_TOKEN:
    try:
        from huggingface_hub import login, whoami
        login(token=HF_TOKEN, add_to_git_credential=False)
        user_info = whoami()
        print(f"✓ Logged in to HuggingFace Hub as: {user_info.get('name', 'unknown')}")
    except Exception as e:
        print(f"⚠️ HF Hub login issue: {e}")
        print("  Will try to use token directly in model loading...")
else:
    print("=" * 60)
    print("⚠️ WARNING: No HF_TOKEN found!")
    print("  Gated models (Gemma, LLaMA) will FAIL!")
    print("  ")
    print("  To fix: Add HF_TOKEN to Colab secrets:")
    print("  1. Click the 🔑 icon in the left sidebar")
    print("  2. Add secret named 'HF_TOKEN' with your token")
    print("  3. Toggle 'Notebook access' ON")
    print("=" * 60)

# ============================================================
# Configure visualization
plt.style.use('seaborn-v0_8-paper')
sns.set_context("talk")

# Global timestamp
TIMESTAMP = datetime.now().strftime("%Y%m%d_%H%M%S")
print(f"\nSession timestamp: {TIMESTAMP}")

print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
# Cell 2: PROMPT SET (Same as Grand Unified Benchmark)

PROMPT_DATASET = {
    "Factual": [
        "The capital city of France is",
        "The atomic number of oxygen is",
        "Water boils at a temperature of",
        "The largest planet in our solar system is",
        "The currency used in Japan is"
    ],
    "Syntactic": [
        "The agreement, which, notwithstanding the fact that it was signed only yesterday, effectively binds all parties immediately, stipulates that",
        "Although the weather was extremely cold, and despite the fact that they had no coats, the children decided to",
        "The professor, having reviewed the complex derivation multiple times without finding the error, finally realized that",
        "To imply that such a fundamental shift in policy could occur without significant public debate is to suggest that",
        "Not only did the experiment fail to yield the expected results, but it also demonstrated that the initial hypothesis was"
    ],
    "Cliche": [
        "The true meaning of happiness is often found in",
        "Actions speak louder than",
        "It is what it is, and we must simply",
        "Time heals all",
        "Life is a journey, not a"
    ],
    "Novel": [
        "The epistemological implications of quantum decoherence suggest that the observer is",
        "If consciousness creates reality, then the paradox of the unobserved electron implies",
        "The intersection of baroque architecture and cybernetic theory creates a space where",
        "Calculating the trajectory of a hyperspace jump requires factoring in the variability of",
        "The symbiotic relationship between fungal mycelium and digital neural networks results in"
    ],
    "Nonsense": [
        "Table sky run blue jump quickly under over",
        "Purple idea furiously sleep colorless green",
        "Clock river dance potato seven fast",
        "Window eat loud tomorrow yellow under",
        "Fish bicycle logic cloud mountain swim"
    ]
}

total_prompts = sum(len(v) for v in PROMPT_DATASET.values())
print(f"Total prompts: {total_prompts} (5 categories x 5 prompts)")

In [ ]:
# Cell 3: TWIN PAIRS CONFIGURATION

TWIN_PAIRS = {
    "Mistral-7B": {
        "base": {
            "hf_path": "mistralai/Mistral-7B-v0.1",
            "name": "Mistral-7B-Base",
            "type": "base"
        },
        "instruct": {
            "hf_path": "mistralai/Mistral-7B-Instruct-v0.2",
            "name": "Mistral-7B-Instruct",
            "type": "instruct"
        },
        "expected_base_gain": 1.11,
        "norm_type": "RMSNorm"
    },
    "Gemma-7B": {
        "base": {
            "hf_path": "google/gemma-7b",
            "name": "Gemma-7B-Base",
            "type": "base"
        },
        "instruct": {
            "hf_path": "google/gemma-7b-it",
            "name": "Gemma-7B-IT",
            "type": "instruct"
        },
        "expected_base_gain": 2.31,
        "norm_type": "RMSNorm"
    },
    "LLaMA-3.1-8B": {
        "base": {
            "hf_path": "meta-llama/Llama-3.1-8B",
            "name": "LLaMA-3.1-8B-Base",
            "type": "base"
        },
        "instruct": {
            "hf_path": "meta-llama/Llama-3.1-8B-Instruct",
            "name": "LLaMA-3.1-8B-Instruct",
            "type": "instruct"
        },
        "expected_base_gain": 1.48,
        "norm_type": "RMSNorm"
    }
}

# Reference from Grand Unified Benchmark
REFERENCE_GAINS = {
    "Pythia-6.9B": 0.80,
    "GPT-J-6B": 1.065,
    "GPT2-XL": 1.02,
    "Mistral-7B": 1.11,
    "LLaMA-3.1-8B": 1.48,
    "Gemma-7B": 2.31
}

print("Twin Pairs to test:")
for family, pair in TWIN_PAIRS.items():
    print(f"\n{family}:")
    print(f"  Base:     {pair['base']['hf_path']}")
    print(f"  Instruct: {pair['instruct']['hf_path']}")
    print(f"  Expected Base Gain: {pair['expected_base_gain']}")

In [ ]:
# Cell 4: MEASUREMENT ENGINE

def get_layer_list(model):
    """Get the transformer layers from different model architectures."""
    # LLaMA/Mistral/Gemma style
    if hasattr(model, 'model') and hasattr(model.model, 'layers'):
        return model.model.layers
    # GPT-2/GPT-J style
    elif hasattr(model, 'transformer') and hasattr(model.transformer, 'h'):
        return model.transformer.h
    # Pythia style
    elif hasattr(model, 'gpt_neox') and hasattr(model.gpt_neox, 'layers'):
        return model.gpt_neox.layers
    else:
        raise ValueError(f"Unknown model architecture: {type(model)}")

def measure_thermodynamics(model, tokenizer, text, device='cuda'):
    """Measure residual stream gain and output entropy."""
    
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    norms = []
    
    def get_norm_hook():
        def hook(module, input, output):
            if isinstance(output, tuple):
                hidden_state = output[0]
            else:
                hidden_state = output
            
            if hasattr(hidden_state, 'last_hidden_state'):
                hidden_state = hidden_state.last_hidden_state
            
            try:
                last_token_norm = float(torch.norm(hidden_state[0, -1]).cpu().item())
                norms.append(last_token_norm)
            except Exception as e:
                print(f"Warning: Could not compute norm: {e}")
        return hook
    
    handles = []
    try:
        layers = get_layer_list(model)
        for layer in layers:
            handles.append(layer.register_forward_hook(get_norm_hook()))
    except Exception as e:
        print(f"Warning: Could not register hooks: {e}")
        return None
    
    try:
        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits
    except Exception as e:
        print(f"Error in forward pass: {e}")
        for h in handles:
            h.remove()
        return None
    
    for h in handles:
        h.remove()
    
    if len(norms) >= 2:
        last_gain = float(norms[-1] / norms[-2]) if norms[-2] > 0 else 1.0
        total_amp = float(norms[-1] / norms[0]) if norms[0] > 0 else 1.0
    else:
        last_gain = 1.0
        total_amp = 1.0
    
    last_token_logits = logits[0, -1, :].float()
    probs = torch.softmax(last_token_logits, dim=0).cpu().numpy().astype(np.float64)
    probs = np.clip(probs, 1e-10, 1.0)
    probs = probs / probs.sum()
    ent = float(entropy(probs))
    
    top_idx = int(torch.argmax(last_token_logits).item())
    top_prob = float(probs[top_idx])
    top_token = tokenizer.decode([top_idx])
    
    return {
        "last_gain": float(last_gain),
        "total_amp": float(total_amp),
        "entropy": float(ent),
        "top_token": str(top_token),
        "top_prob": float(top_prob),
        "n_layers": int(len(norms))
    }

print("Measurement engine ready.")

In [ ]:
# Cell 5: RUN TWIN TEST

all_results = []
twin_summary = {}

print("="*70)
print("TWIN TEST: BASE vs INSTRUCT")
print("Testing Nature vs Nurture Hypothesis")
print("="*70 + "\n")

for family_name, pair_config in TWIN_PAIRS.items():
    print(f"\n{'='*60}")
    print(f"FAMILY: {family_name}")
    print(f"{'='*60}")
    
    family_results = {}
    
    for variant in ["base", "instruct"]:
        model_config = pair_config[variant]
        hf_path = model_config["hf_path"]
        model_name = model_config["name"]
        
        print(f"\n  Loading {model_name}...")
        print(f"    Path: {hf_path}")
        
        try:
            # Load tokenizer
            tokenizer = AutoTokenizer.from_pretrained(
                hf_path,
                token=HF_TOKEN if HF_TOKEN else None
            )
            if tokenizer.pad_token is None:
                tokenizer.pad_token = tokenizer.eos_token
            
            # Load model
            model = AutoModelForCausalLM.from_pretrained(
                hf_path,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                token=HF_TOKEN if HF_TOKEN else None,
                low_cpu_mem_usage=True
            )
            model.eval()
            
            print(f"    Model loaded. Testing {total_prompts} prompts...")
            
            model_results = []
            
            for category, prompts in PROMPT_DATASET.items():
                print(f"      {category}: ", end="")
                for prompt in prompts:
                    res = measure_thermodynamics(model, tokenizer, prompt)
                    
                    if res is not None:
                        result_entry = {
                            "Family": str(family_name),
                            "Model": str(model_name),
                            "Variant": str(variant),
                            "Norm_Type": str(pair_config["norm_type"]),
                            "Category": str(category),
                            "Prompt": str(prompt),
                            "Entropy": float(res["entropy"]),
                            "Last_Gain": float(res["last_gain"]),
                            "Total_Amp": float(res["total_amp"]),
                            "Top_Token": str(res["top_token"]),
                            "Top_Prob": float(res["top_prob"])
                        }
                        all_results.append(result_entry)
                        model_results.append(res["last_gain"])
                        print(".", end="", flush=True)
                    else:
                        print("X", end="", flush=True)
                print(" Done")
            
            # Calculate statistics
            mean_gain = float(np.mean(model_results))
            std_gain = float(np.std(model_results))
            
            family_results[variant] = {
                "mean_gain": mean_gain,
                "std_gain": std_gain,
                "n_samples": len(model_results)
            }
            
            print(f"\n    {model_name}: Mean Gain = {mean_gain:.4f} ± {std_gain:.4f}")
            
            # Cleanup
            del model
            del tokenizer
            torch.cuda.empty_cache()
            gc.collect()
            
        except Exception as e:
            print(f"\n    FAILED: {e}")
            import traceback
            traceback.print_exc()
            family_results[variant] = None
    
    # Store twin summary
    twin_summary[family_name] = family_results
    
    # Compare twins
    if family_results.get("base") and family_results.get("instruct"):
        base_gain = family_results["base"]["mean_gain"]
        instruct_gain = family_results["instruct"]["mean_gain"]
        
        print(f"\n  TWIN COMPARISON ({family_name}):")
        print(f"    Base:     {base_gain:.4f}")
        print(f"    Instruct: {instruct_gain:.4f}")
        print(f"    Delta:    {instruct_gain - base_gain:+.4f}")
        print(f"    Ratio:    {instruct_gain / base_gain:.3f}x")
        
        # Same sign test
        base_sign = "EXPANSION" if base_gain > 1.0 else "DAMPENING" if base_gain < 1.0 else "NEUTRAL"
        inst_sign = "EXPANSION" if instruct_gain > 1.0 else "DAMPENING" if instruct_gain < 1.0 else "NEUTRAL"
        
        if base_sign == inst_sign:
            print(f"    Sign:     SAME ({base_sign}) ✓")
        else:
            print(f"    Sign:     DIFFERENT! Base={base_sign}, Instruct={inst_sign} ✗")

print("\n" + "="*70)
print("TWIN TEST COMPLETE")
print(f"Total measurements: {len(all_results)}")
print("="*70)

In [ ]:
# Cell 6: CREATE DATAFRAME & ANALYZE

df = pd.DataFrame(all_results)

# Define filenames
CSV_FILE = f"twin_test_results_{TIMESTAMP}.csv"
JSON_FILE = f"twin_test_results_{TIMESTAMP}.json"
PNG_MAIN = f"twin_test_results_{TIMESTAMP}.png"
PNG_SCATTER = f"twin_test_scatter_{TIMESTAMP}.png"

# Save raw data
df.to_csv(CSV_FILE, index=False)
print(f"Saved: {CSV_FILE}")

# Summary statistics
print("\n" + "="*70)
print("TWIN TEST RESULTS SUMMARY")
print("="*70)

print("\nMean Gain per Model:")
summary = df.groupby(['Family', 'Model', 'Variant'])['Last_Gain'].agg(['mean', 'std', 'count'])
print(summary.round(4))

print("\nMean Gain per Category (aggregated):")
print(df.groupby(['Variant', 'Category'])['Last_Gain'].mean().unstack().round(4))

In [ ]:
# Cell 7: NATURE VS NURTURE STATISTICAL TEST

print("\n" + "="*70)
print("NATURE VS NURTURE HYPOTHESIS TEST")
print("="*70)

hypothesis_results = []

for family_name in TWIN_PAIRS.keys():
    if family_name not in twin_summary:
        continue
    
    base_data = twin_summary[family_name].get("base")
    inst_data = twin_summary[family_name].get("instruct")
    
    if not base_data or not inst_data:
        continue
    
    base_gain = base_data["mean_gain"]
    inst_gain = inst_data["mean_gain"]
    
    # Same sign test
    base_sign = 1 if base_gain > 1.0 else -1 if base_gain < 1.0 else 0
    inst_sign = 1 if inst_gain > 1.0 else -1 if inst_gain < 1.0 else 0
    same_sign = base_sign == inst_sign
    
    # Calculate metrics
    delta = inst_gain - base_gain
    ratio = inst_gain / base_gain if base_gain != 0 else 1.0
    
    # RLHF effect direction
    if delta > 0.05:
        rlhf_effect = "AMPLIFIES"
    elif delta < -0.05:
        rlhf_effect = "DAMPENS"
    else:
        rlhf_effect = "NEUTRAL"
    
    hypothesis_results.append({
        "Family": family_name,
        "Base_Gain": base_gain,
        "Instruct_Gain": inst_gain,
        "Delta": delta,
        "Ratio": ratio,
        "Same_Sign": same_sign,
        "RLHF_Effect": rlhf_effect
    })
    
    print(f"\n{family_name}:")
    print(f"  Base:     {base_gain:.4f} ({'>' if base_gain > 1 else '<'} 1.0)")
    print(f"  Instruct: {inst_gain:.4f} ({'>' if inst_gain > 1 else '<'} 1.0)")
    print(f"  Delta:    {delta:+.4f}")
    print(f"  Ratio:    {ratio:.3f}x")
    print(f"  Same Sign: {'✓ YES' if same_sign else '✗ NO'}")
    print(f"  RLHF Effect: {rlhf_effect}")

# Overall verdict
print("\n" + "="*70)
print("OVERALL VERDICT")
print("="*70)

all_same_sign = all(r["Same_Sign"] for r in hypothesis_results)
n_pairs = len(hypothesis_results)

if all_same_sign:
    print(f"\n  NATURE DOMINATES: {n_pairs}/{n_pairs} pairs have SAME SIGN")
    print("  → Thermodynamic profile is an architectural 'genetic trait'")
    print("  → RLHF modulates magnitude but does NOT invert sign")
    verdict = "NATURE_DOMINATES"
else:
    n_same = sum(1 for r in hypothesis_results if r["Same_Sign"])
    print(f"\n  MIXED RESULT: {n_same}/{n_pairs} pairs have same sign")
    if n_same >= n_pairs * 0.5:
        print("  → Nature partially dominates")
        verdict = "NATURE_PARTIAL"
    else:
        print("  → Nurture can override nature")
        verdict = "NURTURE_DOMINATES"

print(f"\n  Verdict: {verdict}")

In [ ]:
# Cell 8: VISUALIZATION - Twin Comparison

fig, axes = plt.subplots(2, 2, figsize=(16, 14))

# A. Bar chart: Base vs Instruct per Family
ax1 = axes[0, 0]

families = list(TWIN_PAIRS.keys())
x = np.arange(len(families))
width = 0.35

base_gains = [twin_summary[f]["base"]["mean_gain"] if twin_summary.get(f, {}).get("base") else 0 for f in families]
inst_gains = [twin_summary[f]["instruct"]["mean_gain"] if twin_summary.get(f, {}).get("instruct") else 0 for f in families]

bars1 = ax1.bar(x - width/2, base_gains, width, label='Base', color='#1f77b4', alpha=0.8)
bars2 = ax1.bar(x + width/2, inst_gains, width, label='Instruct', color='#ff7f0e', alpha=0.8)

ax1.axhline(1.0, ls='--', c='red', lw=2, alpha=0.7, label='Neutral (1.0)')
ax1.set_ylabel('Mean Last Layer Gain')
ax1.set_xlabel('Model Family')
ax1.set_title('A. Twin Test: Base vs Instruct')
ax1.set_xticks(x)
ax1.set_xticklabels(families)
ax1.legend()
ax1.grid(True, alpha=0.3, axis='y')

# Add value labels
for bar in bars1:
    height = bar.get_height()
    ax1.annotate(f'{height:.2f}', xy=(bar.get_x() + bar.get_width()/2, height),
                 xytext=(0, 3), textcoords="offset points", ha='center', va='bottom', fontsize=10)
for bar in bars2:
    height = bar.get_height()
    ax1.annotate(f'{height:.2f}', xy=(bar.get_x() + bar.get_width()/2, height),
                 xytext=(0, 3), textcoords="offset points", ha='center', va='bottom', fontsize=10)

# B. Scatter: Base vs Instruct Gain
ax2 = axes[0, 1]

if len(base_gains) > 0 and len(inst_gains) > 0:
    ax2.scatter(base_gains, inst_gains, s=200, c=['#2ca02c', '#d62728', '#9467bd'][:len(families)], 
                edgecolors='black', linewidths=2, zorder=5)
    
    # Add family labels
    for i, family in enumerate(families):
        ax2.annotate(family, (base_gains[i], inst_gains[i]), 
                     xytext=(10, 10), textcoords='offset points', fontsize=11)
    
    # Identity line (Nature = Nurture)
    min_val = min(min(base_gains), min(inst_gains)) * 0.9
    max_val = max(max(base_gains), max(inst_gains)) * 1.1
    ax2.plot([min_val, max_val], [min_val, max_val], 'k--', alpha=0.5, label='Identity (Nature=Nurture)')
    
    # Neutral lines
    ax2.axhline(1.0, ls=':', c='red', alpha=0.3)
    ax2.axvline(1.0, ls=':', c='red', alpha=0.3)
    
    ax2.set_xlabel('Base Model Gain')
    ax2.set_ylabel('Instruct Model Gain')
    ax2.set_title('B. Base vs Instruct Correlation')
    ax2.legend(loc='lower right')
    ax2.grid(True, alpha=0.3)

# C. Box plot: Gain distribution by Variant
ax3 = axes[1, 0]
if len(df) > 0:
    sns.boxplot(data=df, x='Family', y='Last_Gain', hue='Variant', ax=ax3)
    ax3.axhline(1.0, ls='--', c='red', alpha=0.5)
    ax3.set_xlabel('Model Family')
    ax3.set_ylabel('Last Layer Gain')
    ax3.set_title('C. Gain Distribution by Family and Variant')
    ax3.legend(loc='upper right')

# D. Complete hierarchy with twins
ax4 = axes[1, 1]

# Collect all models
all_models = []
for family, results in twin_summary.items():
    if results.get("base"):
        all_models.append((f"{family}\n(Base)", results["base"]["mean_gain"], "base"))
    if results.get("instruct"):
        all_models.append((f"{family}\n(Instruct)", results["instruct"]["mean_gain"], "instruct"))

# Add reference models
for name, gain in REFERENCE_GAINS.items():
    if "Mistral" not in name and "LLaMA" not in name and "Gemma" not in name:
        all_models.append((name, gain, "reference"))

# Sort by gain
all_models_sorted = sorted(all_models, key=lambda x: x[1])

names = [m[0] for m in all_models_sorted]
gains = [m[1] for m in all_models_sorted]
colors = []
for m in all_models_sorted:
    if m[2] == "base":
        colors.append('#1f77b4')
    elif m[2] == "instruct":
        colors.append('#ff7f0e')
    else:
        colors.append('#7f7f7f')

bars = ax4.barh(range(len(names)), gains, color=colors, alpha=0.8)
ax4.axvline(1.0, ls='--', c='red', lw=2, alpha=0.7)
ax4.set_yticks(range(len(names)))
ax4.set_yticklabels(names, fontsize=9)
ax4.set_xlabel('Mean Last Layer Gain')
ax4.set_title('D. Complete Hierarchy\n(Blue=Base, Orange=Instruct, Gray=Reference)')
ax4.grid(True, alpha=0.3, axis='x')

# Value labels
for bar, val in zip(bars, gains):
    ax4.text(val + 0.02, bar.get_y() + bar.get_height()/2, 
             f'{val:.2f}', ha='left', va='center', fontsize=9)

plt.tight_layout()
plt.savefig(PNG_MAIN, dpi=150, bbox_inches='tight')
plt.show()

print(f"\nFigure saved: {PNG_MAIN}")

In [ ]:
# Cell 9: SAVE COMPREHENSIVE JSON

def make_serializable(obj):
    if isinstance(obj, (np.integer, np.floating)):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, np.bool_):
        return bool(obj)
    elif isinstance(obj, dict):
        return {k: make_serializable(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [make_serializable(v) for v in obj]
    return obj

results_json = {
    "experiment": "Twin Test: Base vs Instruct",
    "purpose": "Test Nature vs Nurture hypothesis - is thermodynamic profile genetic?",
    "date": TIMESTAMP,
    "n_families": len(TWIN_PAIRS),
    "n_prompts_per_model": total_prompts,
    "n_total_measurements": len(df),
    
    "twin_pairs_tested": list(TWIN_PAIRS.keys()),
    
    "twin_summary": make_serializable(twin_summary),
    
    "hypothesis_results": make_serializable(hypothesis_results),
    
    "verdict": verdict,
    
    "interpretation": {
        "NATURE_DOMINATES": "Thermodynamic profile (Gain) is an architectural invariant that persists through fine-tuning",
        "NATURE_PARTIAL": "Architecture sets a strong prior, but RLHF can modulate significantly",
        "NURTURE_DOMINATES": "RLHF can fundamentally change the thermodynamic profile"
    }[verdict],
    
    "reference_gains": REFERENCE_GAINS,
    
    "all_results": make_serializable(all_results)
}

with open(JSON_FILE, 'w') as f:
    json.dump(results_json, f, indent=2, default=str)

print(f"Results saved to {JSON_FILE}")

In [ ]:
# Cell 10: FINAL VERDICT

print("\n" + "="*70)
print("TWIN TEST: FINAL VERDICT")
print("="*70)

print(f"""
┌─────────────────────────────────────────────────────────────────────────────┐
│                    TWIN TEST: NATURE vs NURTURE                              │
├─────────────────────────────────────────────────────────────────────────────┤
│                                                                              │
│   Hypothesis: "Thermodynamic profile is a genetic trait"                    │
│                                                                              │
│   If TRUE:  Base and Instruct should have SAME SIGN relative to 1.0         │
│   If FALSE: RLHF can INVERT the thermodynamic behavior                       │
│                                                                              │
├─────────────────────────────────────────────────────────────────────────────┤
│                                                                              │
│   RESULTS:                                                                   │
│                                                                              │
""")

for r in hypothesis_results:
    sign_str = "✓" if r["Same_Sign"] else "✗"
    print(f"│   {r['Family']:15} Base={r['Base_Gain']:.2f}  Instruct={r['Instruct_Gain']:.2f}  {sign_str}       │")

print(f"""
│                                                                              │
├─────────────────────────────────────────────────────────────────────────────┤
│                                                                              │
│   VERDICT: {verdict:20}                                       │
│                                                                              │
│   Interpretation:                                                            │
│   {results_json['interpretation'][:70]:70} │
│                                                                              │
└─────────────────────────────────────────────────────────────────────────────┘
""")

if verdict == "NATURE_DOMINATES":
    print("\n  IMPLICATION FOR PAPER #3:")
    print("  → Thermodynamic profiles ARE genetic traits")
    print("  → RLHF fine-tuning modulates but does NOT override")
    print("  → The Pythia anomaly is ARCHITECTURAL, not training-related")
    print("  → We can claim: 'Gain is determined by architecture, not alignment'")

In [ ]:
# Cell 11: DOWNLOAD ALL RESULTS

print("\n" + "="*70)
print("DOWNLOADING RESULTS")
print("="*70)

files_to_download = [CSV_FILE, JSON_FILE, PNG_MAIN]

print("\nFiles to download:")
for f in files_to_download:
    if os.path.exists(f):
        size = os.path.getsize(f) / 1024
        print(f"  {f} ({size:.1f} KB)")
    else:
        print(f"  {f} (NOT FOUND)")

print("\nStarting downloads...")

for f in files_to_download:
    if os.path.exists(f):
        try:
            files.download(f)
            print(f"  Downloaded: {f}")
        except Exception as e:
            print(f"  FAILED: {f}: {e}")

print("\n" + "="*70)
print("TWIN TEST COMPLETE")
print("="*70)
print("\nNext steps:")
print("  1. If NATURE_DOMINATES: Pythia anomaly is architectural mystery")
print("  2. Add Twin Test results to Paper #3")
print("  3. Formulate: 'Thermodynamic profiles are genetic traits'")